# Aula #26 – Processamento de Linguagem Natural & Análise de Sentimento

# NLP

NLP (ou _Natural Language Processing_ - a sigla em português é PLN, _Processamento de Linguagem Natural_, mas essa sigla também é usada para falar de _Programação Neurolinguística_, então, vamos continuar usando NLP, ok?) é a área relacionada a técnicas para o entendimento de linguagem humana (a linguagem natural). A aplicação de técnicas de NLP tem sido vista em muitos domínios, desde a correção de palavras e sugestão de palavras na barra de busca de sites como o _Google_, até em sistemas mais "sofisticados" como os tradutores automáticos e os _home assistant_ e _smart assistants_, como Siri, Alexa ou Google Home, que podem auxiliar na execução de certas tarefas, como agendar compromissos, embora eles ainda [estejam longe de ser à prova de erros](https://www.nytimes.com/2018/05/25/business/amazon-alexa-conversation-shared-echo.html). Com o crescimento de conteúdo (mais artigos e mais fontes espalhadas), a importância do NLP também tem crescido, pois tarefas como sumarização automática e classificação automática de conteúdo (por ex., para checagem contra _fake news_) são cada vez mais necessárias em nossas vidas.

Nesta aula, vamos focar no uso de algumas técnicas de NLP para processamento de texto. Entretanto, é importante lembrar que NLP não se restringe a textos escritos, podendo ser aplicado também para processamento de fala, como é o caso dos _smart assistants_.

Diferentes tarefas de NLP, em geral, têm como passos iniciais as seguintes etapas:

* pré-processamento do texto (que pode ser uma combinação de diferentes processamentos, envolvendo modificações a nível de palavra e identificação de entidades/funções sintáticas/etc.)

* transformação do texto em quantidades numéricas (tipicamente vetores de números inteiros ou reais)

## Agenda de hoje

* análise de sentimento (detecção de polaridade)

* ferramenta para encontrar ingredientes que combinam (usando _embeddings_ criados por _word2vec_)

## Análise de sentimento

Esse conjunto de dados é composto de reviews de restaurantes e foi modificado a partir do dataset usado no workshop `SemEval` (_International Workshop on Semantic  Evaluation_) na [edição de 2016](http://alt.qcri.org/semeval2016/task5/).

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('data/datasets/restaurants_semeval2016/train.csv')
test_df = pd.read_csv('data/datasets/restaurants_semeval2016/test.csv')

In [3]:
train_df.head()

,text,polarity
0,"It is nearly impossible to get a table, so if ...",positive
1,I won't go back unless someone else is footing...,negative
2,There are so many better places to visit!,negative
3,This place is a must visit!,positive
4,but the service was a bit slow.,positive


In [4]:
test_df.head()

,text,polarity
0,"To be completely fair, the only redeeming fact...",negative
1,I loved it and would HIGHLY RECOMMEND.,positive
2,Nice job!,positive
3,Try the lobster teriyaki and the rose special ...,positive
4,Can’t believe how an expensive NYC restaurant ...,negative


Como podemos ver, tanto no dataset de treino, como no de teste, temos o texto e polaridade da sentença. Nosso objetivo é construir um **classificador de sentimentos**, que recebe uma sentença (referente a um review de restaurante) e é capaz de predizer se o review é positivo ou negativo em relação ao restaurante.

In [5]:
print('\n'.join(train_df.sample(n=10)['text']))

If only they delivered, they'd make a mint!
If you're interested in good tasting (without the fish taste or smell), large portions and creative sushi dishes this is your place...
the food is always fresh ...
And the service was simply spendid - quite a delight.
The scallion pancakes and fried dumplings were nothing out of the ordinary.
We will go back every time we are in the City.
The service is really attentive and charming.
the turkey burgers are scary!
Should check-up on us more frequently, don't you think?
It's a rather cramped and busy restaurant and it closes early.


Nos exemplos de frases acima, podemos ver que as sentenças incluem pontuações, contrações (e.g. `you're` ao invés de `you are`), letras maiúsculas e minúsculas... Seria ideal que conseguíssemos _normalizar_ o texto, de forma a diminuir a quantidade de palavras diferentes.

Um módulo bastante utilizado nesse tipo de tarefa é o `spacy`. No `spacy`, é possível treinar um modelo para que ele reconheça certas estruturas comuns em textos de uma determinada língua. No caso do inglês (e também do português), podemos baixar o modelo e começar a usar!

Além de ser bem completo, ele costuma ser mais rápido que o módulo "concorrente" `nltk`. O [benchmark abaixo](https://blog.thedataincubator.com/2016/04/nltk-vs-spacy-natural-language-processing-in-python/) compara três tarefas (tokenização de palavras, tokenização de sentenças e classificação gramatical - comumente chamado de PoS tag, _part-of-speech tag_). Note que o tempo está em `ms`. Para processar o texto de nosso dataset, não vamos fazer uso do tokenizador de sentenças, logo, para o nosso caso de uso, realmente o `spacy` parece ser a melhor opção!

<img src="data/nb_figs/timing_nltk_spacy_2016.png" width="700"/>

In [6]:
import spacy
import en_core_web_sm

In [ ]:
# baixando o modelo de inglês
#!python -m spacy download en_core_web_sm

In [8]:
# carregando o modelo
# nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

In [9]:
nlp.pipe_names

['tagger', 'parser', 'ner']

**Usando o spacy**

Ao fazer uma chamada do tipo `nlp(text)`, é aplicado o pipeline composto pelo `tagger` (PoS tag), `parser` (_parseador_ de dependências) e `ner` (reconhecimento de entidades) no texto. O retorno é um objeto do tipo `Doc`, que é uma sequência de objetos `Token`.

In [10]:
doc = nlp("It's also attached to Angel's Share, which is a cool, more romantic bar...")

In [11]:
[token for token in doc]

[It,
 's,
 also,
 attached,
 to,
 Angel,
 's,
 Share,
 ,,
 which,
 is,
 a,
 cool,
 ,,
 more,
 romantic,
 bar,
 ...]

### Entidades (named entities)

São termos que se referem a objetos como pessoas, locais, organizações (ex. empresas) etc.

Os tipos de entidades disponíveis no modelo do `spacy` são:

<img src="data/nb_figs/entities_table.png" width="500"/>

Dado um objeto do tipo `Doc`, podemos ver quais são as entidades presentes no texto que ele representa, usando o atributo `ents`, como pode ser visto abaixo.

In [12]:
doc.ents

(Angel's Share,)

Veja que também podemos visualizar as entidades utilizando a ferramenta `displacy`:

In [13]:
spacy.displacy.render(doc, style='ent', jupyter=True)

Como podemos ver, nesse caso, houve a identificação (correta) de `Angel's Share` como uma entidade. Identificar as entidades pode ser útil pois, dependendo do problema, remover entidades do texto pode melhorar o desempenho do seu modelo.

Para as manipulações que queremos fazer, é útil saber um pouco sobre a API do `spacy` para objeto `Token`. Abaixo, temos um excerto selecionado dos métodos e atributos definidos para `Token`.

```python
Help on Token object:

class Token(builtins.object)
 |  An individual token – i.e. a word, punctuation symbol, whitespace,
 |  etc.
 |  
 |  Methods defined here:
 |  
 |  __len__(...)
 |      The number of unicode characters in the token, i.e. `token.text`.
 |      
 |      RETURNS (int): The number of unicode characters in the token.
 |  
 |  __str__(self, /)
 |      Return str(self).

 |  Data descriptors defined here:
 |  
 |  idx
 |      RETURNS (int): The character offset of the token within the parent
 |      document.
 |  
 |  is_alpha
 |      RETURNS (bool): Whether the token consists of alpha characters.
 |      Equivalent to `token.text.isalpha()`.
 |  
 |  is_ascii
 |      RETURNS (bool): Whether the token consists of ASCII characters.
 |      Equivalent to `[any(ord(c) >= 128 for c in token.text)]`.
 |  
 |  is_bracket
 |      RETURNS (bool): Whether the token is a bracket.
 |  
 |  is_currency
 |      RETURNS (bool): Whether the token is a currency symbol.
 |  
 |  is_digit
 |      RETURNS (bool): Whether the token consists of digits. Equivalent to
 |      `token.text.isdigit()`.
 |  
 |  is_lower
 |      RETURNS (bool): Whether the token is in lowercase. Equivalent to
 |      `token.text.islower()`.
 |  
 |  is_punct
 |      RETURNS (bool): Whether the token is punctuation.
 |  
 |  is_space
 |      RETURNS (bool): Whether the token consists of whitespace characters.
 |      Equivalent to `token.text.isspace()`.
 |  
 |  is_stop
 |      RETURNS (bool): Whether the token is a stop word, i.e. part of a
 |      "stop list" defined by the language data.
 |  
 |  is_upper
 |      RETURNS (bool): Whether the token is in uppercase. Equivalent to
 |      `token.text.isupper()`
 |  
 |  lemma_
 |      RETURNS (unicode): The token lemma, i.e. the base form of the word,
 |      with no inflectional suffixes.
 |  
 |  like_email
 |      RETURNS (bool): Whether the token resembles an email address.
 |  
 |  like_num
 |      RETURNS (bool): Whether the token resembles a number, e.g. "10.9",
 |      "10", "ten", etc.
 |  
 |  like_url
 |      RETURNS (bool): Whether the token resembles a URL.
 |
 |  lower_
 |      RETURNS (unicode): The lowercase token text. Equivalent to
 |      `Token.text.lower()`.
 |  
 |  norm_
 |      RETURNS (unicode): The token's norm, i.e. a normalised form of the
 |      token text. Usually set in the language's tokenizer exceptions or
 |      norm exceptions.
 |  
 |  pos_
 |      RETURNS (unicode): Coarse-grained part-of-speech tag.
 |  
 |  sentiment
 |      RETURNS (float): A scalar value indicating the positivity or
 |      negativity of the token.
 |  
 |  shape_
 |      RETURNS (unicode): Transform of the tokens's string, to show
 |      orthographic features. For example, "Xxxx" or "dd".
 |  
 |  string
 |      Deprecated: Use Token.text_with_ws instead.
 |  
 |  subtree
 |      A sequence of all the token's syntactic descendents.
 |      
 |      YIELDS (Token): A descendent token such that
 |          `self.is_ancestor(descendent)`.
```

Sabendo mais sobre a API, vemos que é possível extrair atributos dos tokens, como demonstrado abaixo.

In [14]:
for token in doc:
    print(f'Palavra: "{token}" - (é pontuação: "{token.is_punct}")')

Palavra: "It" - (é pontuação: "False")
Palavra: "'s" - (é pontuação: "False")
Palavra: "also" - (é pontuação: "False")
Palavra: "attached" - (é pontuação: "False")
Palavra: "to" - (é pontuação: "False")
Palavra: "Angel" - (é pontuação: "False")
Palavra: "'s" - (é pontuação: "False")
Palavra: "Share" - (é pontuação: "False")
Palavra: "," - (é pontuação: "True")
Palavra: "which" - (é pontuação: "False")
Palavra: "is" - (é pontuação: "False")
Palavra: "a" - (é pontuação: "False")
Palavra: "cool" - (é pontuação: "False")
Palavra: "," - (é pontuação: "True")
Palavra: "more" - (é pontuação: "False")
Palavra: "romantic" - (é pontuação: "False")
Palavra: "bar" - (é pontuação: "False")
Palavra: "..." - (é pontuação: "True")


### Stopwords

Para muitas tarefas de NLP, é bom prestar atenção nas chamadas _stopwords_, que são as palavras mais comuns que aparecem no texto.

Como assim? Vamos tokenizar (de maneira simplista, mas note que estamos usando expressões regulares. Para refrescar a memória, você pode usar o [pythex](https://pythex.org/)) e imprimir as palavras mais comuns da coluna `text` de nosso dataset de treino.

In [15]:
from collections import Counter

In [16]:
Counter(sum(train_df['text'].str.lower().str.split(r'[\W\s]+').tolist(), [])).most_common(10)

[('', 309),
 ('the', 177),
 ('a', 103),
 ('to', 97),
 ('and', 92),
 ('i', 90),
 ('is', 79),
 ('it', 75),
 ('for', 56),
 ('you', 55)]

Vendo a frequência das palavras no texto, podemos entender de que tipo de palavras estamos falando, certo?

No modelo que temos carregado, podemos obter a lista de stopwords percorrendo o vocabulário do modelo (`nlp.vocab`) e incluindo na lista somente as palavras que são stopwords (ou seja, com `is_stop` igual a `True`).

In [17]:
en_stopwords = sorted([tok.text for tok in nlp.vocab if tok.is_stop])

**_Ponto importante_:** Como nossa tarefa é uma análise de sentimentos, seria muito ruim perder certas _stopwords_ como palavras de negação, afinal, "Eu **não** gosto disso"  é muito diferente de "Eu gosto disso"!

Para facilitar marcar as palavras que gostaríamos de manter no texto (excluindo-as da lista de palavras do modelo do `spacy`), vamos utilizar `widgets`, a partir do módulo [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/index.html). Os `widgets` servem para prover interatividade dentro do notebook, mas vamos logo entender o poder deles com o exemplo abaixo.

In [18]:
from ipywidgets import widgets, HBox, VBox

In [19]:
checkbox = lambda desc, default_value: widgets.Checkbox(
    value=default_value,
    description=desc,
    disabled=False,
)

In [20]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [21]:
excluded = ['almost', 'although', 'anywhere', 'but', 'cannot', 'elsewhere', 'enough', 'every', 'everyone', 'everything', 'everywhere',
            'except', 'however', 'less', 'more', 'never', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'nowhere', 'off',
            'otherwise', 'still', 'though']
items = [checkbox(w, w in set(excluded)) for w in en_stopwords]
all_boxes = []
for chunk_items in chunks(items, 7):
    all_boxes.append(HBox(chunk_items))

**Tarefa:** Marque os boxes das palavras que deveríamos manter no texto (ou seja, que NÃO deveriam ser _stopwords_).

Note que já há algumas palvras marcadas. Fique à vontade para marcar mais ou mesmo desmarcar os boxes.

In [22]:
VBox(all_boxes)

Rode a célula abaixo para efetivar as mudanças no modelo do spacy.

In [23]:
for i, w in enumerate(en_stopwords):
    nlp.vocab[w].is_stop = not items[i].value

E vamos checar, por amostragem, se está tudo certo mesmo.

In [24]:
assert((nlp.vocab['no'].is_stop, nlp.vocab['which'].is_stop) == (False, True))

Excluídas da lista de stopwords:

`['almost', 'although', 'anywhere', 'but', 'cannot', 'elsewhere', 'enough', 'every', 'everyone', 'everything', 'everywhere', 'except', 'however', 'less', 'more', 'never', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'nowhere', 'off', 'otherwise', 'still', 'though']`

### Normalização do texto

Para normalizar os textos de nosso dataset, vamos processar o texto da seguinte forma:

* expandir as contrações (e.g. `don't` -> `do not`);

* remover entidades;

* colocar o texto em letra minúscula (poderia ser em letra maiúscula também, o importante é a consistência!);

* remover pontuação;

* remover _stopwords_.

In [25]:
import re

In [26]:
from utils.contractions import CONTRACTIONS_DICT

In [27]:
CONTRACTIONS_DICT

{"ain't": 'are not',
 "aren't": 'are not',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "'cause": 'because',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he would',
 "he'd've": 'he would have',
 "he'll": 'he will',
 "he'll've": 'he will have',
 "he's": 'he is',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how is',
 "I'd": 'I would',
 "I'd've": 'I would have',
 "I'll": 'I will',
 "I'll've": 'I will have',
 "I'm": 'I am',
 "I've": 'I have',
 "isn't": 'is not',
 "it'd": 'it would',
 "it'd've": 'it would have',
 "it'll": 'it will',
 "it'll've": 'it will have',
 "it's": 'it is',
 "let's": 'let us',
 "ma'am": 'madam',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'mu

**Tarefa**: Crie uma função chamada `expand_contractions` que recebe uma variável do tipo `string` e expande as contrações que eventualmente existirem lá.

Dica: use o dicionário de contrações/expansões `CONTRACTIONS_DICT` e expressões regulares para fazer as substituições. Note que não é necessário que seja mantido o _case_ (a caixa alta/baixa) das palavras, pois vamos colocar todas as palavras em _lowercase_ (caixa baixa). É possível manter o _case_ das palavras e se você estiver curioso sobre isso, sugiro ler mais no [StackOverflow](https://stackoverflow.com/questions/24893977/whats-the-best-way-to-regex-replace-a-string-in-python-but-keep-its-case).

In [28]:
def expand_contractions(text):
    if "'" in text:
        for contracted, expanded in CONTRACTIONS_DICT.items():
            text = re.sub(contracted, expanded, text, flags=re.I)
    return text

In [29]:
assert(expand_contractions("It's").lower() == 'it is')

**Tarefa:** Crie uma função chamada `normalize_text` que recebe uma variável do tipo `string` e retorna uma `string` que:
    1. tem as contrações expandidas;
    2. não possui entidades, pontuação e _stopwords_;
    3. está em letra minúscula.
    
Dica: use a função criada acima para expandir as contrações e use o modelo do spacy carregado `nlp` para te ajudar a fazer as demais normalizações.

**_Ponto importante_:** há um _bug_ na versão `2.0.0` do modelo do spacy (que é a que estamos usando), que faz com que a detecção de _stopwords_ seja _case sensitive_ (ou seja, dependa do estado da palavra, em caixa alta/baixa). Esse _bug_ faz com que somente sejam reconhecidas como _stopwords_ palavras em _lowercase_ (caixa baixa). Para ler mais sobre isso, você pode ler os comentários sobre isso no [github](https://github.com/explosion/spaCy/issues/1889).

Como já foi reforçado antes, em nosso caso, não vamos manter o _case_ das palavras, por isso, para corrigir o comportamento de detecção de _stopwords_, basta que façamos a transformação para _lowercase_ ANTES de checar se a palavra é uma _stopword_, ok?

In [30]:
def remove_ents(text):
    doc = nlp(text)
    for ent in doc.ents:
        text = text.replace(ent.text, '')
    return text

In [31]:
def normalize_text(text):
    text = remove_ents(text)
    if len(text) == 0:
        return text
    doc = nlp(expand_contractions(text).lower())
    new_tokens =[]
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        new_tokens.append(token.text)
    return ' '.join(new_tokens)

**Atenção: A célula abaixo pode dar erro porque o conjunto de stopwords pode ser diferente**

In [32]:
assert(normalize_text("It's also attached to Angel's Share, which is a cool, more romantic bar...") == 'attached cool more romantic bar')

**Tarefa:** Aplique sua função `normalize_text` à coluna `text` (dos dois dataframes, `train_df` e `test_df`), criando uma coluna nova `norm_text`, que será usada para treinarmos um modelo de análise de sentimento.

In [33]:
train_df['norm_text'] = train_df['text'].apply(normalize_text)
test_df['norm_text'] = test_df['text'].apply(normalize_text)

In [34]:
train_df = train_df[train_df.norm_text.str.len() > 0]
test_df = test_df[test_df.norm_text.str.len() > 0]

In [35]:
train_df.sample(n=5)

,text,polarity,norm_text
246,It's somewhere you can eat and be happy.,positive,eat happy
34,"I am so coming back here again, as much as I can.",positive,coming
233,Tiny dessert was $8.00...just plain overpriced...,negative,tiny dessert $ plain overpriced
179,A must try!,positive,try
13,Sunday afternoons there is a band playing and ...,positive,afternoons band playing lots fun


In [36]:
test_df.sample(n=5)

,text,polarity,norm_text
19,A perfect place to take out of town guests any...,positive,perfect place town guests time
3,Try the lobster teriyaki and the rose special ...,positive,try lobster teriyaki rose special roll
50,What a find!,positive,find
79,Can't wait to go back.,positive,not wait
45,"Next time we will go somewhere else, or try ot...",negative,time try restaurants close like 's claim


Você pode (e deve!) rodar as duas células acima algumas vezes para checar os resultados da normalização. Caso não se sinta satisfeito com a normalização feita, volte e edite as _stopwords_ ou sua função de normalização até se sentir satisfeito(a) com os resultados :)

### Modelo (finalmente!!!)

Finalmente, chegamos à parte de treinar o modelo de análise de sentimentos!!!

Ou quase... na verdade, antes de treinar o modelo, precisamos transformar o texto em _features_ numéricas.

A maneira mais simples de transformar um texto em um vetor de números é usando o método comumente chamado de _Bag of words_.

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
examples_for_bow = [
    'camisa preta',
    'botao feito linha preta',
    'considera-se caro preco botao camisa botao',
    'linha costurar botão mesma camisa',
    'costurar linha camisa mesma botao'
]

In [39]:
cv = CountVectorizer(max_features=5, strip_accents='unicode', binary=True)

In [40]:
bow_matrix = cv.fit_transform(examples_for_bow)
bow_matrix

<5x5 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [41]:
bow_matrix.todense()

matrix([[0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=int64)

In [42]:
cv.vocabulary_

{'camisa': 1, 'botao': 0, 'linha': 3, 'costurar': 2, 'mesma': 4}

In [43]:
pd.DataFrame(bow_matrix.todense(), columns=sorted(cv.vocabulary_.items(), key=lambda item: item[1]))

,"(botao, 0)","(camisa, 1)","(costurar, 2)","(linha, 3)","(mesma, 4)"
0,0,1,0,0,0
1,1,0,0,1,0
2,1,1,0,0,0
3,1,1,1,1,1
4,1,1,1,1,1


Note que os exemplos `3` e `4` têm a mesma representação numérica, mesmo que a ordem das palavras não seja a mesma! Essa é uma característica desse método.

#### Treino de classificador _Naive Bayes_

**Tarefa:** Monte um pipeline (criando um objeto do tipo `Pipeline`), que inclua a transformação do texto em _features_ numéricas (`CountVectorizer`) e o classificador `MultinomialNB`. Treine o modelo usando esse pipeline com os dados de input de `train_df`.

Dica: Lembre-se de transformar o _target_ (coluna `polarity`) em 0s e 1s - 0 para `negative` e 1 para `positive`. Se quiser um exemplo, você pode ver esse [post no Medium](https://medium.com/@minbaekim/text-mining-preprocess-and-naive-bayes-classifier-da0000f633b2).

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [45]:
train_df['y'] = (train_df['polarity'] == 'positive').astype(int)
test_df['y'] = (test_df['polarity'] == 'positive').astype(int)

In [46]:
steps = [
    ('vect', CountVectorizer(max_features=200)),
    ('clf', MultinomialNB(fit_prior=False))
]

pipeline = Pipeline(steps)

In [47]:
X_train = train_df['norm_text'].values
y_train = train_df['y'].values

In [48]:
sentiment_analyzer = pipeline.fit(X_train, y_train)

**Tarefa:** Teste seu classificador usando um texto de exemplo. Veja que você pode usar tanto o método `predict` como o método `predict_proba`.

In [49]:
test_df.iloc[0]

text         To be completely fair, the only redeeming fact...
polarity                                              negative
norm_text    completely fair redeeming factor food average ...
y                                                            0
Name: 0, dtype: object

In [50]:
sentiment_analyzer.predict([test_df.iloc[0]['norm_text']])

array([0])

In [51]:
sentiment_analyzer.predict_proba([test_df.iloc[0]['norm_text']])

array([[ 0.71908733,  0.28091267]])

#### Avaliação do classificador

**Tarefa:** Faça a predição da coluna `norm_text` e compare o resultado com o vetor target (coluna `polarity` em 0s e 1s).

Dica: Imprima o [classification_report](https://scikit-learn.org/0.19/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report) e a [matriz de confusão](https://scikit-learn.org/0.19/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix).

In [52]:
X_test = test_df['norm_text'].values
y_test = test_df['y'].values

In [53]:
y_pred = sentiment_analyzer.predict(X_test)

In [54]:
from sklearn.metrics import classification_report, confusion_matrix

In [55]:
print(classification_report(y_test, y_pred, target_names=['negative', 'positive']))

             precision    recall  f1-score   support

   negative       0.51      0.78      0.62        23
   positive       0.89      0.70      0.78        57

avg / total       0.78      0.72      0.74        80



In [56]:
confusion_matrix(y_test, y_pred)

array([[18,  5],
       [17, 40]])

** O que você achou do classificador? Ele é bom ou ruim?**

#### Mais considerações e possíveis modificações

Note que após aplicar o _Bag of words_, também poderíamos ter aplicado a transformação Tfidf, que discrimina as palavras de acordo com a "relevância" delas em cada documento em relação ao _corpus_ (i.e. o conjunto total de documentos).

**Tarefa Bônus:** Inclua no _pipeline_ a transformação `Tfidf` e compare os resultados.

Vimos que nosso classificador possui tanto o método `predict` como o método `predict_proba`. Ao usar o método `predict_proba`, temos as probabilidades de que o texto seja `positive` ou `negative`. Podemos escolher um _threshold_ de forma a maximizar uma das métricas.

**Tarefa Bônus:** Escolha esse threshold de forma a maximizar nosso `recall` de exemplos positivos.

Outra alternativa seria processar o texto de uma maneira diferente: imagine que só quiséssemos incluir três tipos de classes gramaticais: verbo, adjetivo e advérbio, que normalmente são as classes que indicam o sentimento de um indivíduo em relação a alguma coisa.

No modelo do `spacy`, é possível identificar os seguintes tipos de PoS tag:

* ADJ
* ADP
* ADV
* CONJ
* DET
* INTJ
* NOUN
* PART (e.g. possessive marker _'s_)
* PRON
* PROPN
* PUNCT
* SPACE
* SYM
* VERB
* X (unknown)

**Obs.:** na verdade isso vale para o _coarse-grained PoS tag_ (que é visto chamando o atributo `pos` de um objeto `Token`), para o _fine-grained PoS tag_, há muito [mais classificações disponíveis](https://spacy.io/api/annotation).

In [57]:
doc = nlp("the food is always fresh ...")

In [58]:
for token in doc:
    print(token, token.pos_)

the DET
food NOUN
is VERB
always ADV
fresh ADJ
... PUNCT


Assim, se imprimíssemos somente as palavras de classe `ADJ`, `ADV` e `VERB`, teríamos:

In [59]:
allowed_pos = set(['ADJ', 'ADV', 'VERB'])
for token in doc:
    if token.pos_ not in allowed_pos:
        continue
    print(token, token.pos_)

is VERB
always ADV
fresh ADJ


**Tarefa Bônus:** Filtre os adjetivos, advérbios e verbos do texto e construa um novo modelo que use esse novo texto como input e avalie o resultado.